In [170]:
import pandas as pd
import numpy as np
import datetime

In [171]:
data = pd.read_csv("order_brush_order.csv")  # 读取csv
data["event_time"] = pd.to_datetime(data["event_time"])  # 转换成日期格式
data.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [189]:
finalList = pd.DataFrame(columns=["shopid", "userid"])
finalList

,shopid,userid


In [190]:
groupedByShop = data.groupby(["shopid"])
for shopid, group in groupedByShop:
    # blackList = ""
    # for index, row in group.iterrows():
    #     st = row.event_time
    #     unit = group[(st <= group['event_time']) & (group['event_time'] < st + datetime.timedelta(hours=1))]
    #     trsAmt = unit.shape[0]
    #     UsrAmt = len(unit["userid"].unique())
    #     # if (trsAmt/UsrAmt) >= 3:
    #         # print(unit["userid"].value_counts().shape[0])
    #         # print("------------------------")
    if blackList == "":
        finalList =finalList.append([{"shopid": shopid,"userid": 0}],ignore_index=True)
print(finalList)
# print(finalList)
        # print(unit)
        # print("trans:   ",trsAmt)
        # print("usr: :   ",UsrAmt)